In [ ]:
import json
import os
from pprint import pprint
import boto3
from pandas.io.json import json_normalize
import pandas as pd
import time

bucket = 'yelpadsproject'
folder = 'data'

cols = ['review_id', 'user_id', 'business_id','text','stars','sentiment', 'mixed', 'negative', 'neutral', 'positive']
lst = []

cols2 = ['business_id','name']
bussdf = []

# get the region (pay attention to the region of your S3, SageMaker ... keep them in same region where possible)
region = boto3.Session().region_name

# create the s3 client
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key=f'data/yelp_academic_dataset_review_8K.json')
file_content = obj['Body'].read().decode('ISO-8859-1') 


bussobj = s3.get_object(Bucket=bucket, Key=f'data/yelp_academic_dataset_business.json')
buss_content = bussobj['Body'].read().decode('utf-8') 

print("Process Businesses")
for buss in buss_content.split("\n"):
    if len(buss) > 0:
        buss_content = json.loads(buss)
        business_id = buss_content['business_id']
        name = buss_content['name']
        bussdf.append([business_id,name])

df2 = pd.DataFrame(bussdf, columns=cols2)

comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')

print("Process Reviews")
for kv in file_content.split("\n"):
    json_content = json.loads(kv)
    review_id = json_content['review_id']
    user_id = json_content['user_id']
    business_id = json_content['business_id']
    stars = json_content['stars']
    
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    
    sent_text = json_content['text'].replace("|","")
    sentiment_text = sent_text.replace("\n", " ")
    
    # remove punctuation from the string
    no_punct = ""
    for char in sent_text:
       if char not in punctuations:
           no_punct = no_punct + char

    # display the unpunctuated string
   
    compResponse = comprehend.detect_sentiment(Text=sentiment_text, LanguageCode='en')
    sentiment = compResponse['Sentiment']
    mixed = compResponse['SentimentScore']['Mixed']
    negative = compResponse['SentimentScore']['Negative']
    neutral = compResponse['SentimentScore']['Neutral']
    positive = compResponse['SentimentScore']['Positive']
    
    lst.append([review_id, user_id, business_id, sentiment_text,stars, sentiment, mixed, negative, neutral, positive])

print ("Creating File.....")
df1 = pd.DataFrame(lst, columns=cols)
    
outputset = pd.merge(df1, df2, on='business_id', how='inner')

prefix = "output"
ts = str(int(time.time()))

filenameString = "aws_comprehend_" + ts + ".txt"
                          
dirfilename = "output/"+ filenameString

outputset.to_csv(dirfilename, sep='|')

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, filenameString)).upload_file(dirfilename)
print("file created")


Process Businesses
Process Reviews


In [63]:
total_rec = 0
postive_corr = 0
negative_corr = 0


cols3 = ['yelpStars', 'comprehendScore', 'comprehendText']
graphData = []

for index, row in outputset.iterrows():
    total_rec = total_rec + 1
    stars = row['sentiment']
    comprehendText = row['stars']
    if row['stars'] == "POSITIVE":
        comprehendScore = row['positive']
        if stars < 4:
            negative_corr = negative_corr + 1
        else:
            postive_corr = postive_corr + 1
    if row['stars'] == "NEGATIVE":
        comprehendScore = row['negative']
        if stars > 2:
            negative_corr = negative_corr + 1
        else:
            postive_corr = postive_corr + 1

    if row['stars'] == "MIXED":
        comprehendScore = row['mixed']
        if stars != 2 or stars != 4:
            negative_corr = negative_corr + 1
        else:
            postive_corr = postive_corr + 1
            
    if row['stars'] == "NEUTRAL":
        comprehendScore = row['neutral']
        if stars != 3:
            negative_corr = negative_corr + 1
        else:
            postive_corr = postive_corr + 1
            
    graphData.append([stars,comprehendScore, comprehendText])

    
graphData = pd.DataFrame(graphData, columns=cols3)
    
print(total_rec)

print("{0:.1%}".format(negative_corr/total_rec))
print("{0:.1%}".format(postive_corr/total_rec))



8000
17.0%
83.0%


In [106]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#"{0:.0f}%".format(1/3 * 100)

sortedDF = graphData.groupby(['comprehendText','yelpStars'])['comprehendText'].count()
print(sortedDF)


pd.options.display.float_format = '{:.2f}%'.format
pd.crosstab(graphData['comprehendText'],graphData['yelpStars']).apply(lambda r: (r/r.sum())*100, axis=1)




comprehendText  yelpStars
MIXED           1              18
                2              66
                3             132
                4              44
                5               7
NEGATIVE        1            1168
                2             456
                3             165
                4              27
                5              23
NEUTRAL         1              64
                2              37
                3              29
                4              36
                5              60
POSITIVE        1             105
                2             144
                3             435
                4            1471
                5            3513
Name: comprehendText, dtype: int64


yelpStars,1,2,3,4,5
comprehendText,,,,,
MIXED,6.74%,24.72%,49.44%,16.48%,2.62%
NEGATIVE,63.51%,24.80%,8.97%,1.47%,1.25%
NEUTRAL,28.32%,16.37%,12.83%,15.93%,26.55%
POSITIVE,1.85%,2.54%,7.67%,25.95%,61.98%
